https://aheritianad.github.io/teaching/ankatso/mafi/exercises/solutions/20230511

## 2023-05-11


1. `lambda` function
2. `@` decorator
3. bits
   1. size
   2. `&` and
   3. `|` or
   4. `^` xor
   5. `~` neg
   6. `>>` right shift (`//2`)
   7. `<<` left shift (`*2`)
4. `float`ing point number
5. `encrypt` with translation and permutation
6. `rsa` encryption
7. `bcbcpy.crypto`
8. `is` vs `==`

### `lambda` and `@`

In [1]:
def begin(func):
    def _func(*args, **kwargs):
        print(f"begin {func.__name__}")
        out = func(*args, **kwargs)
        return out
    _func.__name__ = func.__name__
    return _func

def end(func):
    def _func(*args, **kwargs):
        out = func(*args, **kwargs)
        print(f"end {func.__name__}")
        return out
    _func.__name__ = func.__name__
    return _func


In [2]:
@begin  # f = begin(lambda n: n**2)
def f(n: int):
    return n**2


@end  # g = end(g)
def g(n: int):
    m = n - 1
    k = n + 1
    return m * k

@begin
@end
def h(n: int):
    return n - 1
# h = begin(end(lambda n: n- 1))

In [3]:
out = f(2)
print(f"f(2) = {out}")

begin f
f(2) = 4


In [4]:
out = g(2)
print(f"g(2) = {out}")

end g
g(2) = 3


In [5]:
out = h(2)
print(f"h(2) = {out}")

begin h
end h
h(2) = 1


In [6]:
def decorator_of_decorators(deco):
    def _deco(func):
        print(f"decorator_of_decorators is decorating {func.__name__} with decorator {deco.__name__}.")
        _func = deco(func)
        return _func
    _deco.__name__ = deco.__name__
    return _deco


@decorator_of_decorators
def new_deco(func):
    def _func(*args, **kwargs):
        out = func(*args, **kwargs)
        print(f"Function {func.__name__} is decorated by new_deco.")
        return out
    
    _func.__name__ = func.__name__
    return _func

In [7]:
@new_deco
def g(n):
    return n

decorator_of_decorators is decorating g with decorator new_deco.


In [8]:
g(6)

Function g is decorated by new_deco.


6

In [9]:
@new_deco
@end
@begin
def g(n):
    return n

decorator_of_decorators is decorating g with decorator new_deco.


In [10]:
g(1)

begin g
end g
Function g is decorated by new_deco.


1

In [11]:
def append_0_to_all_list_argument(func):
    def _func(*args, **kwargs):
        output = func(*args,**kwargs)
        for arg in args:
            if isinstance(arg, list):
                arg.append(0)
        for kwarg in kwargs.values():
            if isinstance(kwarg, list):
                kwarg.append(0)
        return output
    _func.__name__ = func.__name__
    return _func

@append_0_to_all_list_argument
def sum_element(lst1, base,lst2):
    return sum(lst1) + base + sum(lst2)

In [12]:
a = [-1,-2,-3]
b = 4
c = [1,2,3]

sum_element(a,b,c)

4

In [13]:
a,b,c

([-1, -2, -3, 0], 4, [1, 2, 3, 0])

### Bits

#### `&` (and)

11: 1011  
07: 0111  
............  
03: 0011

In [14]:
11&7

3

#### `|` (or)

11: 1011  
07: 0111  
............  
15: 1111

In [15]:
11|7

15

#### `^` (xor)

11: 1011  
07: 0111  
............  
12: 1100

In [16]:
11^7

12

#### `~` (neg)

For $a > 0$, $-a = (\sim a) + 1 \implies \sim a = -a - 1$.  
For $a < 0$, $-a = (\sim a) - 1 \implies \sim a = -a + 1$.  

+11: 00001011  
......................  
-12: 11110100

In [17]:
~11

-12

In [18]:
~-12

11

#### `>>k` (`//`$2^k$)

In [19]:
15//4, 15>>2

(3, 3)

#### `<<k` ($\times 2^k$)

In [20]:
3*2**2, 3<<2

(12, 12)

### Floating point

In [21]:
0.1 + 0.1 == 0.2

True

In [22]:
0.1 + 0.2 == 0.3

False

In [23]:
0.1 + 0.2

0.30000000000000004

**Q**: What??????

Answer: 👉 (https://docs.python.org/3/tutorial/floatingpoint.html/)


In [24]:
def bin_dec(x, precision=100):
    # x = n.d (int.decimal)
    n = int(x)
    d = x - n
    counter = 0
    decimal = ""    
    while d != 0:
        if len(decimal) > precision:
            decimal += "..."
            break
        d = 2*d
        leading = int(d)
        decimal += str(leading)
        d = d - leading
        counter += 1
    
    return bin(n)[2:] + "."  + decimal + f"    ({counter})"

In [25]:
bin_dec(.1)

'0.0001100110011001100110011001100110011001100110011001101    (55)'

In [26]:
eval(repr(0.1)) == 0.1

True

In [27]:
bin_dec(.2)

'0.001100110011001100110011001100110011001100110011001101    (54)'

In [28]:
bin_dec(.3)

'0.010011001100110011001100110011001100110011001100110011    (54)'

In [29]:
bin_dec(.1+.2)

'0.0100110011001100110011001100110011001100110011001101    (52)'

In [30]:
bin_dec(0.30000000000000004)

'0.0100110011001100110011001100110011001100110011001101    (52)'

In [31]:
print(f"""
.1\t:\t{bin_dec(.1)}
.2\t:\t{bin_dec(.2)}
.1+.2\t:\t{bin_dec(.1+.2)}
.3\t:\t{bin_dec(.3)}
.30...4\t:\t{bin_dec(0.30000000000000004)}
"""
)


.1	:	0.0001100110011001100110011001100110011001100110011001101    (55)
.2	:	0.001100110011001100110011001100110011001100110011001101    (54)
.1+.2	:	0.0100110011001100110011001100110011001100110011001101    (52)
.3	:	0.010011001100110011001100110011001100110011001100110011    (54)
.30...4	:	0.0100110011001100110011001100110011001100110011001101    (52)



IEEE floating point standard 754:
===

$$(-1)^s\times 2^{E} \times M$$  

Bits representation: [$s$ | $exp$ | $frac$ ] where $exp$ is an `unsigned int`.

- $s$: $1$ bit for sign,
- $bias = 2^{\text{bit\_size\_of}(exp) - 1} - 1$ 
- if $exp = 0$:
  - $M = 0.[frac]$ (denormalized or near zero),
  - $E = 1 + exp - bias = 1 - bias$.
- else if $exp < 11...1$:
  - $M = 1.[frac]$ (normalized),
  - $E = exp - bias$
- else: ($exp = 11...1$)
  - `inf` for $frac = 0$ and
  - `NaN` for $frac \neq 0$

Types:
===

- Single precision (`float32`):
  - $frac$: $23$ bits,
  - $exp$: $8$ bits ($\implies bias = 2^{8-1} - 1 = 127$).
- Half precision (`float16`):
  - $frac$: $10$ bits,
  - $exp$: $5$ bits.
- Double precision (`float64`):
  - $frac$: $52$ bits,
  - $exp$: $11$ bits.
- Quadruple precision (`float128`):
  - $frac$: $112$ bits,
  - $exp$: $15$ bits.
- Octuple precision (`float256`):
  - $frac$: $236$ bits,
  - $exp$: $19$ bits.


### `==` vs `is`

In [32]:
a = (1,2)
b = a
c = (1,2)

In [33]:
a == b, b == c, c == a

(True, True, True)

In [34]:
a is b, b is c, c is a

(True, False, False)